In [15]:
import sys
print(sys.path)
sys.path.append("/home/aspr/lib/python3.9/site-packages")
import openai

In [17]:
openai.api_key = ... # HERE goes your API key

In [19]:
openai.File.create(file=open("data/datasets/codex_context.jsonl"), purpose='answers')

<File file id=file-Fr9KmMv8fFJ3VlhOOPsCfzUV at 0x7f70d5df1310> JSON: {
  "bytes": 228,
  "created_at": 1638883016,
  "filename": "context.jsonl",
  "id": "file-Fr9KmMv8fFJ3VlhOOPsCfzUV",
  "object": "file",
  "purpose": "answers",
  "status": "uploaded",
  "status_details": null
}

In [20]:
examples = [["Concatenate elements of a list 'x' of multiple integers to a single integer", "sum(d * 10 ** i for i, d in enumerate(x[::-1]))"], ["BeautifulSoup search string 'Elsie' inside tag 'a'", "soup.find_all('a', string='Elsie')"], ["find rows matching `(0,1)` in a 2 dimensional numpy array `vals`", "np.where((vals == (0, 1)).all(axis=1))"]]

In [66]:
examples_hs = [["NAME_BEGIN Booty Bay Bodyguard NAME_END ATK_BEGIN 5 ATK_END DEF_BEGIN 4 DEF_END COST_BEGIN 5 COST_END DUR_BEGIN -1 DUR_END TYPE_BEGIN Minion TYPE_END PLAYER_CLS_BEGIN Neutral PLAYER_CLS_END RACE_BEGIN NIL RACE_END RARITY_BEGIN Common RARITY_END DESC_BEGIN Taunt DESC_END", "class BootyBayBodyguard(MinionCard):\n\ndef __init__(self):\n    super().__init__('Booty Bay Bodyguard', 5, CHARACTER_CLASS.ALL, CARD_RARITY.COMMON)\n\ndef create_minion(self, player):\n    return Minion(5, 4, taunt=True)"], ["NAME_BEGIN Darkscale Healer NAME_END ATK_BEGIN 4 ATK_END DEF_BEGIN 5 DEF_END COST_BEGIN 5 COST_END DUR_BEGIN -1 DUR_END TYPE_BEGIN Minion TYPE_END PLAYER_CLS_BEGIN Neutral PLAYER_CLS_END RACE_BEGIN NIL RACE_END RARITY_BEGIN Common RARITY_END DESC_BEGIN Battlecry : Restore 2 Health to all friendly characters . DESC_END", "class DarkscaleHealer(MinionCard):\ndef __init__(self):\n    super().__init__('Darkscale Healer', 5, CHARACTER_CLASS.ALL, CARD_RARITY.COMMON, battlecry=Battlecry(Heal(2), CharacterSelector()))\n\ndef create_minion(self, player):\n    return Minion(4, 5)"]]

In [69]:
ans = openai.Answer.create(
    search_model="ada",
    model="davinci",
    question="NAME_BEGIN Frostwolf Warlord NAME_END ATK_BEGIN 4 ATK_END DEF_BEGIN 4 DEF_END COST_BEGIN 5 COST_END DUR_BEGIN -1 DUR_END TYPE_BEGIN Minion TYPE_END PLAYER_CLS_BEGIN Neutral PLAYER_CLS_END RACE_BEGIN NIL RACE_END RARITY_BEGIN Common RARITY_END DESC_BEGIN Battlecry : Gain +1/+1 for each other friendly minion on the battlefield . DESC_END",
    documents=[],
    examples_context="Python 3 HearthStone class code generation",
    examples=examples_hs,
    max_rerank=10,
    max_tokens=250,
    stop=["\n", "<|endoftext|>"]
)

In [70]:
print(ans)

{
  "answers": [
    "class FrostwolfWarlord(MinionCard):"
  ],
  "completion": "cmpl-4Ctm69vRco21VOPr4w5Mn7Q3nLxt5",
  "model": "davinci:2020-05-03",
  "object": "answer",
  "search_model": "ada",
  "selected_documents": []
}


In [32]:
all_snippets = json.load(open("./metrics-evaluation/to-grade/all-singles.json"))

In [37]:
for i, item in enumerate(all_snippets):
    print(i)
    ans = openai.Answer.create(
        search_model="ada",
        model="davinci",
        question=item["intent"],
        documents=[],
        examples_context="Python 3 code generation",
        examples=examples,
        max_rerank=10,
        max_tokens=50,
        stop=["\n", "<|endoftext|>"]
    )
    item["codex"] = ans["answers"][0]
    print(item)

0
{'intent': 'send a signal `signal.SIGUSR1` to the current process', 'snippet': 'os.kill(os.getpid(), signal.SIGUSR1)', 'baseline': "os.system('<unk>.png',s = 300)", 'tranx-annot': 'sys.signal(`signal.SIGUSR1`)', 'best-tranx': 'os.system(`signal.SIGUSR1`)', 'best-tranx-rerank': 'os.system(`< unk > < unk > < unk >`)', 'codex': 'os.kill(os.getpid(), signal.SIGUSR1)'}
1
{'intent': "decode a hex string '4a4b4c' to UTF-8.", 'snippet': "bytes.fromhex('4a4b4c').decode('utf-8')", 'baseline': '"""\\\\ascii.decode(\'utf-8\').decode(\'utf-8\')', 'tranx-annot': 'print(4a4b4c.decode(`utf-8`).decode(`utf-8`))', 'best-tranx': 'print(4a4b4c.decode(`4a4b4c`,`ignore`))', 'best-tranx-rerank': 'print(4a4b4c.decode(`4a4b4c`))', 'codex': "'4a4b4c'.decode('utf-8')"}
2
{'intent': 'check if all elements in list `myList` are identical', 'snippet': 'all(x == myList[0] for x in myList)', 'baseline': '[int(i)for i in range(100)]', 'tranx-annot': 'all(isinstance(list,my List)for x in my List)', 'best-tranx': '[i f

KeyError: 'intent'

In [38]:
all_snippets

[{'intent': 'send a signal `signal.SIGUSR1` to the current process',
  'snippet': 'os.kill(os.getpid(), signal.SIGUSR1)',
  'baseline': "os.system('<unk>.png',s = 300)",
  'tranx-annot': 'sys.signal(`signal.SIGUSR1`)',
  'best-tranx': 'os.system(`signal.SIGUSR1`)',
  'best-tranx-rerank': 'os.system(`< unk > < unk > < unk >`)',
  'codex': 'os.kill(os.getpid(), signal.SIGUSR1)'},
 {'intent': "decode a hex string '4a4b4c' to UTF-8.",
  'snippet': "bytes.fromhex('4a4b4c').decode('utf-8')",
  'baseline': '"""\\\\ascii.decode(\'utf-8\').decode(\'utf-8\')',
  'tranx-annot': 'print(4a4b4c.decode(`utf-8`).decode(`utf-8`))',
  'best-tranx': 'print(4a4b4c.decode(`4a4b4c`,`ignore`))',
  'best-tranx-rerank': 'print(4a4b4c.decode(`4a4b4c`))',
  'codex': "'4a4b4c'.decode('utf-8')"},
 {'intent': 'check if all elements in list `myList` are identical',
  'snippet': 'all(x == myList[0] for x in myList)',
  'baseline': '[int(i)for i in range(100)]',
  'tranx-annot': 'all(isinstance(list,my List)for x in m

In [39]:
misha_snippets = json.load(open("./metrics-evaluation/to-grade/all-singles.json"))
for misha, codex in zip(misha_snippets, all_snippets):
    misha["codex"] = codex["codex"]

In [40]:
misha_snippets

[{'intent': 'send a signal `signal.SIGUSR1` to the current process',
  'snippet': 'os.kill(os.getpid(), signal.SIGUSR1)',
  'baseline': "os.system('<unk>.png',s = 300)",
  'tranx-annot': 'sys.signal(`signal.SIGUSR1`)',
  'best-tranx': 'os.system(`signal.SIGUSR1`)',
  'best-tranx-rerank': 'os.system(`< unk > < unk > < unk >`)',
  'grade-tranx-annot': 1,
  'grade-snippet': 4,
  'grade-best-tranx': 1,
  'grade-best-tranx-rerank': 0,
  'grade-baseline': 1,
  'codex': 'os.kill(os.getpid(), signal.SIGUSR1)'},
 {'intent': "decode a hex string '4a4b4c' to UTF-8.",
  'snippet': "bytes.fromhex('4a4b4c').decode('utf-8')",
  'baseline': '"""\\\\ascii.decode(\'utf-8\').decode(\'utf-8\')',
  'tranx-annot': 'print(4a4b4c.decode(`utf-8`).decode(`utf-8`))',
  'best-tranx': 'print(4a4b4c.decode(`4a4b4c`,`ignore`))',
  'best-tranx-rerank': 'print(4a4b4c.decode(`4a4b4c`))',
  'grade-best-tranx-rerank': 1,
  'grade-baseline': 1,
  'grade-tranx-annot': 1,
  'grade-best-tranx': 1,
  'grade-snippet': 4,
  'co

In [41]:
with open("./metrics-evaluation/to-grade/all-singles.json", 'w') as f:
    json.dump(misha_snippets, f)

In [42]:
i = 0
for item in misha_snippets:
    for key in item.keys():
        if "grade" in key:
            i += 1
print(i)

2500


In [50]:
dima_snippets = json.load(open("/home/aspr/IdeaProjects/metrics-codegen/to-grade/conala/all-singles.seryozha.json"))
for dima, codex in zip(dima_snippets, misha_snippets):
    dima["codex"] = codex["codex"]
with open("/home/aspr/IdeaProjects/metrics-codegen/to-grade/conala/all-singles.seryozha.json", 'w') as f:
    json.dump(dima_snippets, f)

In [62]:
x = [True, False, True]
sum(1 << i for i, b in enumerate(x) if b)
sum(x[i] * 2 ** i for i in range(len(x)))


5